### Data Science Job Market: <i>Year 2020</i>

This project improves upon a previous data initiave by bringing in <i>updated data</i> to more precisely reflect current opportunities in the data science job market with the added variable of the ongoing COVID-19 global pandemic. Our goals include:
   * Determine the top 3 or top 10 technical skills that are in-demand for aspiring candidates
   * Deduce the hottest geographic areas that are doing high volume hiring in the data science space 
   * Summarize our observations and subsequent conclusions in a comprehensive jobs report which candidates can use as a <i>"ready-reckoner"</i> to make informed choices

#### Import Dependencies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

from collections import Counter

#### Data Preprocessing

<blockquote> Datasets were obtained from Kaggle and are separated by job functions <i>viz.,</i> data <i>analyst</i>, data <i>engineer</i> and data <i>scientist</i></blockquote>

In [2]:
# Set identifiers for the 3 CSV files
a_file = "Resources/data_analyst_jobs.csv"
e_file = "Resources/data_engineer_jobs.csv"
s_file = "Resources/data_scientist_jobs.csv"

In [3]:
# Read CSV files into respective dataframes 
analyst_df = pd.read_csv(a_file)
engineer_df = pd.read_csv(e_file)
scientist_df = pd.read_csv(s_file)

In [4]:
# Drop useless columns, rename them first
analyst_df.rename(columns={'Unnamed: 0': 'Nameless'}, inplace=True)
del analyst_df['Nameless']

scientist_df.rename(columns={'Unnamed: 0': 'Nameless'}, inplace=True)
del scientist_df['Nameless']

# Delete 'index' column from the data scientist jobs dataframe
del scientist_df['index']

<blockquote> Executive decisions were made to keep all data points for 
<i>engineering</i> roles since data engineers tend to wear many hats ranging from "tester"
to "architect". The engineering jobs mixed in with the other two job roles i.e., data 
analyst and data scientist were filtered out.</blockquote>

In [6]:
# Split values in 'Job Title' to create 'Job Category' that reflects the defined job function
engineer_df['Job Category'] = engineer_df['Job Title'].apply(lambda x: 'Data Engineer' if 'Engineer' in x else 'Data Engineer')
    
# Additionally, filter out engineering roles mixed in with analyst/scientist functions
analyst_df['Job Category'] = analyst_df['Job Title'].apply(lambda x: 'Data Analyst' if 'Analyst' in x else '')
scientist_df['Job Category'] = scientist_df['Job Title'].apply(lambda x: 'Data Scientist' if 'Scientist' in x else '')

In [7]:
# Combine the 3 dataframes into a single dataframe
datadf_list = [analyst_df, engineer_df, scientist_df]
datadf = pd.concat(datadf_list, join="inner")

<blockquote>The choice to drop the <i>Competitors</i> column from the combined dataframe stems from a reasoning that every aspiring candidate can do their own level of research about a Company and its competitors. From a very <i>practical</i> standpoint, we think this particular column is irrelevant to our objectives</blockquote>

In [8]:
# Drop the 'Competitors' column from the combined dataframe
del datadf['Competitors']

In [9]:
# Create a key-value mapping to rename columns
new_cols = {'Job Title':'Job Title', 'Job Category':'Job Type',
            'Salary Estimate':'Salary Range', 'Job Description':'Job Description',
            'Rating':'Job Rating', 'Company Name':'Company Name',
            'Location':'Location', 'Headquarters':'HQ',
            'Size':'Company Size',  'Founded':'Founded In',
            'Type of ownership':'Ownership Type', 'Industry':'Job Industry',
            'Sector':'Job Sector', 'Revenue':'Company Revenue','Easy Apply':'Easy Apply'}

# Rename all columns for the new dataframe
datadf_copy = datadf.copy()
datadf_copy.rename(columns=new_cols, inplace=True)

In [10]:
# Create a new dataframe containing all the data 
dsjobs_df = pd.DataFrame(datadf_copy, columns=['Job Title', 'Job Type',
                                          'Salary Range', 'Job Description',
                                          'Job Rating', 'Company Name',
                                          'Location', 'HQ', 'Company Size',
                                          'Founded In', 'Ownership Type',
                                          'Job Industry', 'Job Sector',
                                          'Company Revenue', 'Easy Apply'])

# Set the index for the new dataframe
dsjobs_df.set_index(['Job Title'])

# Display first 5 records of the combined dataframe
dsjobs_df.head()

,Job Title,Job Type,Salary Range,Job Description,Job Rating,Company Name,Location,HQ,Company Size,Founded In,Ownership Type,Job Industry,Job Sector,Company Revenue,Easy Apply
0,"Data Analyst, Center on Immigration and Justic...",Data Analyst,$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,3.2,Vera Institute of Justice\n3.2,"New York, NY","New York, NY",201 to 500 employees,1961,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD),True
1,Quality Data Analyst,Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,3.8,Visiting Nurse Service of New York\n3.8,"New York, NY","New York, NY",10000+ employees,1893,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1
2,"Senior Data Analyst, Insights & Analytics Team...",Data Analyst,$37K-$66K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,3.4,Squarespace\n3.4,"New York, NY","New York, NY",1001 to 5000 employees,2003,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,-1
3,Data Analyst,Data Analyst,$37K-$66K (Glassdoor est.),Requisition NumberRR-0001939\nRemote:Yes\nWe c...,4.1,Celerity\n4.1,"New York, NY","McLean, VA",201 to 500 employees,2002,Subsidiary or Business Segment,IT Services,Information Technology,$50 to $100 million (USD),-1
4,Reporting Data Analyst,Data Analyst,$37K-$66K (Glassdoor est.),ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,3.9,FanDuel\n3.9,"New York, NY","New York, NY",501 to 1000 employees,2009,Company - Private,Sports & Recreation,"Arts, Entertainment & Recreation",$100 to $500 million (USD),True


#### Data Transformation
<blockquote> This step involves operations such as handling missing values or values with '-1', dropping unwanted columns, creating new columns by splitting string values and finally splitting our main dataframe by type <i>viz.,</i> <b> positions, industry and location </b></blockquote>

##### Clean-up & Splitting Columns

In [11]:
# Clean up the 'Company Name' column to have relevant data 
company_names = dsjobs_df['Company Name'].str.split("\n", n=1, expand=True)
dsjobs_df['Company Name'] = company_names

# Split up 'Location' and 'HQ' into its respective City & State columns
jobs_city_state = dsjobs_df['Location'].str.split(", ", n=1, expand=True)
dsjobs_df['Location City'] = jobs_city_state[0]
dsjobs_df['Location State']= jobs_city_state[1]

hq_city_state = dsjobs_df['Location'].str.split(", ", n=1, expand=True)
dsjobs_df['HQ City'] = hq_city_state[0]
dsjobs_df['HQ State']= hq_city_state[1]

# Eliminate rows that do not have a uniform state code 
non_us_locs = ['Arapahoe, CO', 'United Kingdom']

dsjobs_df = dsjobs_df[~dsjobs_df['Location State'].isin(non_us_locs)]
dsjobs_df = dsjobs_df[~dsjobs_df['HQ State'].isin(non_us_locs)]

# Drop the 'Location' and 'HQ' columns that are no longer needed
del dsjobs_df['Location']
del dsjobs_df['HQ']

##### Eliminating rows with '-1' or 'Unknown' values
<blockquote> The choice to eliminate missing or null values is justified in the objective of narrowing down the job search to those positions from which we can derive the most information. After all, <i>"little knowledge is dangerous!"</i></blockquote>

In [12]:
# Eliminate rows that have missing or unknown values
incomplete_vals = ['-1', 'Unknown', '']

dsjobs_df = dsjobs_df[~dsjobs_df['Job Industry'].isin(incomplete_vals)]
dsjobs_df = dsjobs_df[~dsjobs_df['Job Sector'].isin(incomplete_vals)]
dsjobs_df = dsjobs_df[~dsjobs_df['Ownership Type'].isin(incomplete_vals)]

# Create a key-value mapping to rename all values in 'Ownership Type' column
ownership_dict = {'Nonprofit Organization':'Non-Profit', 'Company - Private':'Private',
                  'Subsidiary or Business Segment':'Subsidiary', 'Company - Public':'Public', 
                  'Hospital':'Hospital', 'Contract':'Contracted', 'Government':'Government',
                  'School / School District':'School', 'Other Organization':'Other',
                  'College / University':'University', 'Self-employed':'Freelance', 
                  'Private Practice / Firm':'Firm', 'Franchise':'Franchise'}

# Replace all the values with relevant strings using the mapping
dsjobs_df = dsjobs_df.replace({"Ownership Type": ownership_dict})

<blockquote> For the <i>Easy Apply</i> column, we make assumptions that '-1' indicates a falsity since not enough information is available to make any other categories </blockquote>

In [13]:
# Replace all '-1' values in 'Easy Apply' with False 
easy_apply_dict = {-1:False}
dsjobs_df = dsjobs_df.replace({"Easy Apply": easy_apply_dict})

##### Forming regular expressions to handle range values
<blockquote> For the columns named <i>'Salary Range', 'Company Size' and 'Company Revenue'</i> we will form custom regular expressions to transform them into intuitive labels</blockquote>

In [ ]:
# Further clean-up of isolated values 
# clean_salary_range = dsjobs_df.loc[dsjobs_df['Salary Estimate'].str.contains(' to 1')] = ''

# clean_salary_range = dsjobs_df.loc[dsjobs_df['Salary Estimate'].str.contains('71K to 130K(Employer')] = '71K to 130K'
# clean_salary_range = dsjobs_df.loc[dsjobs_df['Salary Estimate'].str.contains('82K to 129K(Employer')]= '82K to 129K'
# clean_salary_range = dsjobs_df.loc[dsjobs_df['Salary Estimate'].str.contains('82K to 130K(Employer')] = '82K to 130K'
# clean_salary_range = dsjobs_df.loc[dsjobs_df['Salary Estimate'].str.contains('200K to 250K(Employer')] = '200K to 250K'

# clean_salary_range = dsjobs_df.loc[dsjobs_df['Salary Estimate'].str.contains('34 to 53')] = '34K to 53K'
# clean_salary_range = dsjobs_df.loc[dsjobs_df['Salary Estimate'].str.contains('10 to 26')] = '10K to 26K'
# clean_salary_range = dsjobs_df.loc[dsjobs_df['Salary Estimate'].str.contains('17 to 27')] = '17K to 27K'

# dsjobs_df['Salary Estimate'] = clean_salary_range

# Sort the values in 'Salary Estimate' by ascending order
# dsjobs_df = dsjobs_df.sort_values('Salary Estimate')